HW5
2a,b,c
Monte Carlo (Repetition of 2a,b,c 50 times, all estimates averaged)

We assume Monte Carlo means, to simply repeat the procedure 50 times

In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_score

train = pd.read_csv('Train_Data.csv',index_col = 0)
test = pd.read_csv('Test_Data.csv',index_col = 0)
whole = pd.concat([train,test],axis=0)
whole.reset_index(drop=True,inplace=True)
features = whole.drop(columns=['Family','Genus','Species'])
label_class = whole.iloc[:,[22,23,24]]
                   
                   

hamming_loss_vector = pd.DataFrame(index=np.arange(1,51),columns=['Hamming Loss'])
hamming_distance_vector = pd.DataFrame(index=np.arange(1,51),columns=['Hamming Distance'])
hamming_score_vector = pd.DataFrame(index=np.arange(1,51),columns=['Hamming Score'])
                   
for mc in np.arange(0,50):
    
    print(mc)  # to keep iteration track
                   
    sil = pd.DataFrame(index=np.arange(2,51),columns=['Silhouette_Score_Average'])
    for i in np.arange(2,51):
        kmeans = KMeans(n_clusters=i).fit(features)
        pred_clusters = kmeans.labels_
        sil.iloc[i-2] = silhouette_score(features,pred_clusters)

    max_sil = sil.max(axis=0)
    max_indx = np.argmax(sil['Silhouette_Score_Average'].to_numpy().flatten())
    k_selected = max_indx + 2

    kmeans = KMeans(n_clusters = k_selected).fit(features)
    cluster_groups = kmeans.labels_

    cluster_triplet_df = pd.DataFrame(index = np.arange(0,k_selected),columns=['Fam_Clus','Gen_Clus','Spec_Clus']) # has cluster label details
    for q in np.arange(0,k_selected):
        for l in np.arange(0,3):
            loc_build = []
            for p in np.arange(0,features.shape[0]):
                if cluster_groups[p] == q:
                    loc_build.append(label_class.iloc[p,l])

            build = np.array(loc_build)
            build_counts = np.bincount(build)
            highest_freq = np.argmax(build_counts)
            cluster_triplet_df.iloc[q,l] = highest_freq


    cluster_predictions_label_class = pd.DataFrame(index = np.arange(0,features.shape[0]),columns = ['Family','Genus','Species'])

    for i in np.arange(0,features.shape[0]):
        cluster_predictions_label_class.iloc[i,0] = cluster_triplet_df.iloc[cluster_groups[i],0]
        cluster_predictions_label_class.iloc[i,1] = cluster_triplet_df.iloc[cluster_groups[i],1]
        cluster_predictions_label_class.iloc[i,2] = cluster_triplet_df.iloc[cluster_groups[i],2]


    mis = 0
    for i in np.arange(0,features.shape[0]):
        for j in np.arange(0,3):
            if cluster_predictions_label_class.iloc[i,j] != label_class.iloc[i,j]:
                mis = mis + 1


    hamming_loss = (mis/(3*features.shape[0]))*100
    
    hamming_loss_vector.iloc[mc,0] = hamming_loss               

    hamming_distance = mis/features.shape[0]
    
    hamming_distance_vector.iloc[mc,0] = hamming_distance


    hs = 0
    for i in np.arange(0,features.shape[0]):

        sam_loc = 0
        for j in np.arange(0,3):

            if cluster_predictions_label_class.iloc[i,j] == label_class.iloc[i,j]:  # note the equality here, true calculated
                sam_loc = sam_loc + 1

        pred_loc = cluster_predictions_label_class.iloc[i,:].to_numpy().flatten()
        true_loc = label_class.iloc[i,:].to_numpy().flatten()

        uni_rep_strng = np.concatenate((pred_loc,true_loc)).flatten()  # union, absolute, with repetitions

        uni_bin = np.bincount(uni_rep_strng.astype(int))

        for m in np.arange(0,uni_bin.size):
            if uni_bin[m] > 1:
                uni_bin[m] = 1

        dist_cnt = np.sum(uni_bin)    # count of total distinct elements

        hs_loc = sam_loc/dist_cnt

        hs = hs + hs_loc          # running sum of HS


    hamming_score = (hs/features.shape[0])*100
    
    hamming_score_vector.iloc[mc,0] = hamming_score
                   
            

print('The hamming loss vector for all iterations, in %, is : \n',hamming_loss_vector)

print('The hamming distance vector for all iterations is : \n',hamming_distance_vector)
                   
print('The hamming score/accuracy score vector for all iterations, in %, is : \n',hamming_score_vector)                   

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
The hamming loss vector for all iterations, in %, is : 
    Hamming Loss
1       22.2423
2       22.2423
3       22.2423
4       22.2469
5       22.2423
6       22.2423
7       22.2423
8       22.2423
9       22.2423
10      22.2423
11      22.2423
12      22.2423
13      22.1635
14      22.2423
15      22.1635
16      22.2423
17      24.5124
18      22.1635
19      22.2423
20      22.2423
21      22.2423
22      24.5263
23      22.2423
24      22.2423
25      22.2006
26      22.1774
27      22.2423
28      22.2423
29      22.2423
30      22.2423
31      22.2423
32      24.5263
33      22.2423
34      22.2423
35      22.2284
36      22.2469
37      22.1635
38      22.2423
39      22.2423
40      22.2423
41      22.2423
42      22.2423
43      22.1774
44      22.2423
45      24.5124
46      22.2423
47      22.2423
48      22.2423
49      22.2423
50 

As can be seen, the values change very little through all 50 Monte-Carlo iterations

In [7]:
avg_hamming_loss = hamming_loss_vector.mean(axis=0)
avg_hamming_distance = hamming_distance_vector.mean(axis=0)
avg_hamming_score = hamming_score_vector.mean(axis=0)

print('The average hamming loss over 50 iterations is in % : \n',avg_hamming_loss)
print('The average hamming distance over 50 iterations is : \n',avg_hamming_distance)
print('The average hamming score/accuracy over 50 iterations is in %: \n',avg_hamming_score)

The average hamming loss over 50 iterations is in % : 
 Hamming Loss    22.41464
dtype: float64
The average hamming distance over 50 iterations is : 
 Hamming Distance    0.672439
dtype: float64
The average hamming score/accuracy over 50 iterations is in %: 
 Hamming Score    80.299736
dtype: float64
